### 라이브러리

In [ ]:
# !pip install -q "transformers>=4.42.0" "accelerate>=0.30.0" bitsandbytes peft datasets torchvision pillow mlflow
import torch, transformers, mlflow
print("torch", torch.__version__, "cuda", torch.cuda.is_available())

In [ ]:
# # 데이터 예시
# sample = {
#   "image": "/path/to/photo.jpg",  # 얼굴 이미지 1장
#   "diagnosis_name": "acne",       # ResNet/analysis_result 라벨
#   "health_profile": "height 172cm, weight 66kg, activity moderate, goal: cut 0.5kg/week",
#   "diet_recommendation_meta": "rec_id=123, memo=초기 상담, created_at=2024-01-01",
#   "rules": "허용: 저당, 오메가3 / 제한: 고당, 포화지방 / 조건: 1식 550kcal",
#   "calorie_plan": 550,
#   "diet_json": [
#     {"menuName": "연어 샐러드", "description": "훈제연어+채소", "calories": 520, "notes": "오메가3", "recipeUrl": "..."},
#     {"menuName": "두부 비빔밥", "description": "현미, 두부, 채소", "calories": 560, "notes": "저당", "recipeUrl": "..."},
#   ]
# }


SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' (2419650234.py, line 1)

### 1) 모델/프로세서(Qwen3-VL-7B , 4bit)

In [ ]:
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

base_model = "Qwen/Qwen3-VL-7B-Instruct"  # VRAM 맞춰 조정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

processor = AutoProcessor.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()


## 2) LoRA 준비

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

## 3) 데이터/콜레이터
- 샘플 리스트로 교체

In [ ]:
from datasets import Dataset
from PIL import Image

def build_prompt(record):
    sys_prompt = (
        "You are a dietitian VLM. See the meal image and propose 3-4 diet options as JSON array. "
        "Keys: menuName, description, calories, notes, recipeUrl(optional), skincareUrl(optional). "
        "Calories integers (kcal)."
    )
    user_prompt = (
        f"Image caption: {record['caption']}\n"
        f"Backup label: {record['resnet_label']}\n"
        f"Health info: {record['health_info']}\n"
        f"Rules: {record['rules']}\n"
        f"Calorie target per meal: {record['calorie_plan']} kcal\n"
        "Return JSON only."
    )
    return sys_prompt, user_prompt

def collate_fn(batch):
    images = [Image.open(x["image"]).convert("RGB") for x in batch]
    messages_list = []
    for x, img in zip(batch, images):
        sys_prompt, user_prompt = build_prompt(x)
        messages_list.append([
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": [{"type": "image", "image": img}, {"type": "text", "text": user_prompt}]},
        ])
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    text_inputs = processor.apply_chat_template(messages_list, add_generation_prompt=False,
                                                return_tensors="pt", padding=True)
    labels = text_inputs["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    return {
        "input_ids": text_inputs["input_ids"],
        "attention_mask": text_inputs["attention_mask"],
        "pixel_values": pixel_values,
        "labels": labels,
    }

# TODO: synthesize_sample() 등으로 채운 리스트 사용
train_samples = []  # e.g., [{"image": "...", "caption": "...", ...}, ...]
train_dataset = Dataset.from_list(train_samples)


## 4) MLflow 설정 + Trainer

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers.integrations import MLflowCallback

mlflow.set_experiment("qwen2vl-qlora")
# mlflow.set_tracking_uri("http://your-mlflow:5000")  # 필요 시

training_args = TrainingArguments(
    output_dir="./qlora-qwen2vl2b",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=5,
    save_steps=50,
    save_total_limit=2,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    report_to="mlflow",           # 자동 로깅
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    callbacks=[MLflowCallback()],  # 선택적이지만 추천
)


## 5) 학습 + 로깅

In [ ]:
with mlflow.start_run():
    # 수동으로도 남기고 싶은 파라미터/메모 추가
    mlflow.log_params({
        "lora_r": lora_config.r,
        "lora_alpha": lora_config.lora_alpha,
        "train_batch": training_args.per_device_train_batch_size,
        "grad_accum": training_args.gradient_accumulation_steps,
        "base_model": base_model,
    })
    trainer.train()
    # 어댑터/체크포인트 기록
    save_dir = "qlora-adapter"
    trainer.save_model(save_dir)
    mlflow.log_artifacts(save_dir)


## 6) 추론용 어댑터 로드

In [ ]:
from peft import PeftModel
base_infer = AutoModelForVision2Seq.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
peft_infer = PeftModel.from_pretrained(base_infer, "qlora-adapter")
peft_infer.eval()